In [1]:
# NBVAL_IGNORE_OUTPUT
import lineapy
%load_ext lineapy

x = 100
y = 10

z = [x]
art = lineapy.save(z, "z")
pipeline_dir = lineapy.to_pipeline([art.name], framework='AIRFLOW', output_dir="~/airflow/dags")

RuntimeError: No context set. This could be because the LineaPy extension has not been loaded in your runtime environment. To load the extension, you need to launch your environment with the `lineapy` command, e.g., `lineapy jupyter notebook`. Or, to load the extension without relaunching the environment, execute `%load_ext lineapy` at the top of your session. Check https://docs.lineapy.org/en/main/guides/interfaces.html#jupyter-and-ipython for more detailed instructions.

In [ ]:
# NBVAL_IGNORE_OUTPUT
module_path = pipeline_dir/"z_module.py"
print(module_path.read_text())

In [ ]:
dag_path = pipeline_dir/"z_dag.py"
print(dag_path.read_text())

In [ ]:
# Cleanup pipeline files
import shutil
shutil.rmtree(pipeline_dir)